In [1]:
# 1. Start with your opening balance
opening_cash = 884.70

# 2. Define your lumpy schedule using a dictionary/DataFrame
payroll_schedule = {4: 3500, 8: 3500, 12: 3500}
ad_spend_schedule = {5: 5000, 10: 5000}

# 3. Create the forecast loop
for week in range(1, 14):
    cash_in = 1500 # Base weekly revenue
    cash_out = 500

    # Apply the Lumpy Logic
    if week in payroll_schedule:
        cash_out += payroll_schedule[week]

    net_flow = cash_in - cash_out
    opening_cash += net_flow
    # ... store results in a DataFrame ...

In [3]:
import pandas as pd
import numpy as np

# --- 1. CORE ASSUMPTIONS & STARTING BALANCE ---
STARTING_CASH = 50000  # Initial cash balance (e.g., after last fundraising round)
WEEKS = 13

# --- 2. GROWTH CURVE FLOWS (The Lumpy Logic) ---
# Define specific, non-average, scheduled weekly flows
flow_data = {
    'Income (Subscription/Sales)': [6000, 5000, 6500, 5500, 8000, 6000, 7000, 8500, 9000, 7500, 8000, 10000, 9500],

    # Large collection hits on Week 5 and Week 12
    'Change AR (Collections)': [1000, 1500, 1000, 2000, 4500, 1000, 1500, 2000, 1000, 2500, 1000, 5000, 1500],

    # Payroll hits every 4 weeks (Wk 4, 8, 12)
    'Payroll/Salary': [0, 0, 0, 15000, 0, 0, 0, 16000, 0, 0, 0, 17000, 0],

    # Large Ad Spend hits on Week 4 and 10
    'Direct Fees (Ad Spend)': [500, 700, 1000, 8000, 500, 700, 1000, 500, 700, 8500, 500, 700, 1000],

    # Other Admin/Operating Expenses - relatively stable
    'Other Admin Expense': [1500, 1500, 1500, 1500, 1500, 1600, 1600, 1600, 1600, 1700, 1700, 1700, 1700],

    # AP Payments - intentionally heavy in Week 5 to create the squeeze
    'Change in AP (Payments)': [2000, 2500, 2000, 2000, 7000, 2000, 2500, 2000, 2000, 2500, 2000, 2500, 2000]
}

# --- 3. BUILD AND CALCULATE THE FORECAST ---

# Create the initial DataFrame
df_forecast = pd.DataFrame(flow_data, index=[f'Week {i}' for i in range(1, WEEKS + 1)]).T

# Add Calculation Rows
df_forecast.loc['Total Cash In'] = df_forecast.loc[['Income (Subscription/Sales)', 'Change AR (Collections)']].sum()
df_forecast.loc['Total Cash Out'] = df_forecast.loc[['Payroll/Salary', 'Direct Fees (Ad Spend)', 'Other Admin Expense', 'Change in AP (Payments)']].sum()
df_forecast.loc['Net Cash Flow'] = df_forecast.loc['Total Cash In'] - df_forecast.loc['Total Cash Out']

# Add Balance Rows and Calculate the Rolling Balance
balances = [STARTING_CASH]
closing_cash = STARTING_CASH

for week in range(WEEKS):
    opening_cash = closing_cash
    net_flow = df_forecast.loc['Net Cash Flow'][f'Week {week+1}']
    closing_cash = opening_cash + net_flow

    balances.append(opening_cash)

# Insert the calculated balances back into the DataFrame
df_forecast.loc['Opening Cash Balance'] = balances[:-1]
df_forecast.loc['Closing Cash Balance'] = balances[1:]

# Final clean-up and formatting
df_final = df_forecast.reindex([
    'Opening Cash Balance',
    'Total Cash In',
    'Income (Subscription/Sales)',
    'Change AR (Collections)',
    'Total Cash Out',
    'Payroll/Salary',
    'Direct Fees (Ad Spend)',
    'Other Admin Expense',
    'Change in AP (Payments)',
    'Net Cash Flow',
    'Closing Cash Balance'
]).round(0).astype(int)

# Use Markdown for display (ideal for Colab/GitHub README)
print(df_final.T.to_markdown())

|         |   Opening Cash Balance |   Total Cash In |   Income (Subscription/Sales) |   Change AR (Collections) |   Total Cash Out |   Payroll/Salary |   Direct Fees (Ad Spend) |   Other Admin Expense |   Change in AP (Payments) |   Net Cash Flow |   Closing Cash Balance |
|:--------|-----------------------:|----------------:|------------------------------:|--------------------------:|-----------------:|-----------------:|-------------------------:|----------------------:|--------------------------:|----------------:|-----------------------:|
| Week 1  |                  50000 |            7000 |                          6000 |                      1000 |             4000 |                0 |                      500 |                  1500 |                      2000 |            3000 |                  50000 |
| Week 2  |                  50000 |            6500 |                          5000 |                      1500 |             4700 |                0 |                      7

In [4]:
# Install Plotly Express (usually pre-installed in Colab)
# !pip install plotly-express

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, HTML

In [5]:
def create_cashflow_df(starting_cash):
    # --- Re-run your data and calculation logic here ---

    # Using the Lumpy Data from the previous response for this example:
    flow_data = {
        'Income (Subscription/Sales)': [6000, 5000, 6500, 5500, 8000, 6000, 7000, 8500, 9000, 7500, 8000, 10000, 9500],
        'Change AR (Collections)': [1000, 1500, 1000, 2000, 4500, 1000, 1500, 2000, 1000, 2500, 1000, 5000, 1500],
        'Payroll/Salary': [0, 0, 0, 15000, 0, 0, 0, 16000, 0, 0, 0, 17000, 0],
        'Direct Fees (Ad Spend)': [500, 700, 1000, 8000, 500, 700, 1000, 500, 700, 8500, 500, 700, 1000],
        'Other Admin Expense': [1500, 1500, 1500, 1500, 1500, 1600, 1600, 1600, 1600, 1700, 1700, 1700, 1700],
        'Change in AP (Payments)': [2000, 2500, 2000, 2000, 7000, 2000, 2500, 2000, 2000, 2500, 2000, 2500, 2000]
    }

    df_forecast = pd.DataFrame(flow_data, index=[f'Week {i}' for i in range(1, 14)]).T

    df_forecast.loc['Total Cash In'] = df_forecast.loc[['Income (Subscription/Sales)', 'Change AR (Collections)']].sum()
    df_forecast.loc['Total Cash Out'] = df_forecast.loc[['Payroll/Salary', 'Direct Fees (Ad Spend)', 'Other Admin Expense', 'Change in AP (Payments)']].sum()
    df_forecast.loc['Net Cash Flow'] = df_forecast.loc['Total Cash In'] - df_forecast.loc['Total Cash Out']

    balances = [starting_cash]
    closing_cash = starting_cash

    for week in range(13):
        opening_cash = closing_cash
        net_flow = df_forecast.loc['Net Cash Flow'][f'Week {week+1}']
        closing_cash = opening_cash + net_flow
        balances.append(opening_cash)

    df_forecast.loc['Opening Cash Balance'] = balances[:-1]
    df_forecast.loc['Closing Cash Balance'] = balances[1:]

    df_final = df_forecast.reindex([
        'Opening Cash Balance',
        'Total Cash In',
        'Income (Subscription/Sales)',
        'Change AR (Collections)',
        'Total Cash Out',
        'Payroll/Salary',
        'Direct Fees (Ad Spend)',
        'Other Admin Expense',
        'Change in AP (Payments)',
        'Net Cash Flow',
        'Closing Cash Balance'
    ]).round(0).astype(int)

    return df_final.T # Transpose for easier plotting (Weeks as index)

# Generate the data
df_dashboard = create_cashflow_df(starting_cash=50000)

In [6]:
# Cash Runway (Approximation: Total Cash / Avg Weekly Burn)
avg_net_flow = df_dashboard['Net Cash Flow'].mean()
avg_net_out = df_dashboard.loc[df_dashboard['Net Cash Flow'] < 0, 'Net Cash Flow'].mean()

# Handle positive flow (i.e., you are profitable) or zero flow
if avg_net_flow >= 0 or np.isnan(avg_net_out):
    runway_weeks = "Infinite (Profitable)"
else:
    runway_weeks = int(df_dashboard['Opening Cash Balance'].iloc[0] / abs(avg_net_out))

# Other KPIs
min_cash = df_dashboard['Closing Cash Balance'].min()
cash_conversion_ratio = df_dashboard['Total Cash In'].sum() / df_dashboard['Total Cash Out'].sum()

# Display Tickers using HTML for a better dashboard look
html_kpis = f"""
    <div style="display: flex; justify-content: space-around; padding: 20px; background-color: #f8f9fa; border-radius: 8px;">
        <div style="text-align: center;">
            <h4 style="color: #4CAF50;">Cash Runway</h4>
            <p style="font-size: 1.5em; font-weight: bold;">{runway_weeks} Weeks</p>
        </div>
        <div style="text-align: center;">
            <h4 style="color: {'#FF5722' if min_cash < 20000 else '#2196F3'};">Minimum Cash (Week)</h4>
            <p style="font-size: 1.5em; font-weight: bold;">${min_cash:,.0f} ({df_dashboard['Closing Cash Balance'].idxmin()})</p>
        </div>
        <div style="text-align: center;">
            <h4 style="color: #00BCD4;">Cash Conversion Ratio (In/Out)</h4>
            <p style="font-size: 1.5em; font-weight: bold;">{cash_conversion_ratio:.2f}x</p>
        </div>
    </div>
"""
display(HTML(html_kpis))

In [7]:
fig_balance = px.line(
    df_dashboard,
    y='Closing Cash Balance',
    title='13-Week Closing Cash Balance (Risk Monitor)',
    height=400
)
# Highlight the minimum cash point
min_cash_index = df_dashboard['Closing Cash Balance'].idxmin()
fig_balance.add_trace(go.Scatter(
    x=[min_cash_index],
    y=[min_cash],
    mode='markers',
    marker=dict(color='Red', size=12),
    name=f'Min Cash: ${min_cash:,.0f}'
))
fig_balance.update_layout(xaxis_title="Week", yaxis_title="Cash Balance ($)")
fig_balance.show()

In [8]:
fig_net_flow = px.bar(
    df_dashboard,
    y='Net Cash Flow',
    title='Weekly Net Cash Flow',
    color=np.where(df_dashboard['Net Cash Flow'] >= 0, 'Inflow', 'Outflow'),
    color_discrete_map={'Inflow': '#4CAF50', 'Outflow': '#F44336'},
    height=400
)
fig_net_flow.update_layout(xaxis_title="Week", yaxis_title="Net Flow ($)")
fig_net_flow.show()

In [9]:
print("## 13-Week Forecast Detail Table (All Line Items)")
print(df_dashboard.to_markdown())

## 13-Week Forecast Detail Table (All Line Items)
|         |   Opening Cash Balance |   Total Cash In |   Income (Subscription/Sales) |   Change AR (Collections) |   Total Cash Out |   Payroll/Salary |   Direct Fees (Ad Spend) |   Other Admin Expense |   Change in AP (Payments) |   Net Cash Flow |   Closing Cash Balance |
|:--------|-----------------------:|----------------:|------------------------------:|--------------------------:|-----------------:|-----------------:|-------------------------:|----------------------:|--------------------------:|----------------:|-----------------------:|
| Week 1  |                  50000 |            7000 |                          6000 |                      1000 |             4000 |                0 |                      500 |                  1500 |                      2000 |            3000 |                  50000 |
| Week 2  |                  50000 |            6500 |                          5000 |                      1500 |           